In [ ]:
# Perform git clone as follows

!git clone https://github.com/WEIRDLabUW/cse542_sp24_hw2.git
!cp -r cse542_sp24_hw2/* .

# !NOTE!: Once you are done, copy your implementation of policy gradient, actor critic and
# in the notebook here back to the python script
# when submiting your code

!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf
!pip install setuptools==65.5.0 "wheel<0.40.0"
!pip install gym==0.19.0
!pip install gymnasium==0.29.1
!pip install gymnasium-robotics[mujoco-py]
!pip install gym-notices==0.0.8
!pip install matplotlib
!pip install mujoco
!pip install free-mujoco-py
!pip install pybullet
import os
os.environ['LD_PRELOAD']=':/usr/lib/x86_64-linux-gnu/libGLEW.so'
import os
import torch
import numpy as np
from torch import nn
from torch import optim
from IPython.display import clear_output
import argparse
import collections
import functools
import math
import time
import random
from typing import Any, Callable, Dict, Optional, Sequence, List
import gym
import mujoco_py
from gym import utils
from gym.envs.mujoco import mujoco_env
import torch.nn.functional as F
import copy
from typing import Tuple, Optional, Union
import matplotlib.pyplot as plt


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
import torch
import torch.nn as nn

import math
import copy
import numpy as np
import pickle
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch import distributions as pyd
import torch.optim as optim
from torch.distributions import Categorical
import random

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ReplayBuffer(object):
    """Buffer to store environment transitions."""

    def __init__(self, obs_size, action_size, capacity, device):
        self.capacity = capacity
        self.device = device

        self.obses = np.empty((capacity, obs_size), dtype=np.float32)
        self.next_obses = np.empty((capacity, obs_size), dtype=np.float32)
        self.actions = np.empty((capacity, action_size), dtype=np.float32)
        self.rewards = np.empty((capacity, 1), dtype=np.float32)
        self.not_dones = np.empty((capacity, 1), dtype=np.float32)

        self.idx = 0
        self.last_save = 0
        self.full = False

    def __len__(self):
        return self.capacity if self.full else self.idx

    def add(self, obs, action, reward, next_obs, done):
        idxs = np.arange(self.idx, self.idx + obs.shape[0]) % self.capacity
        self.obses[idxs] = copy.deepcopy(obs)
        self.actions[idxs] = copy.deepcopy(action)
        self.rewards[idxs] = copy.deepcopy(reward)
        self.next_obses[idxs] = copy.deepcopy(next_obs)
        self.not_dones[idxs] = 1.0 - copy.deepcopy(done)

        self.full = self.full or (self.idx + obs.shape[0] >= self.capacity)
        self.idx = (self.idx + obs.shape[0]) % self.capacity

    def sample(self, batch_size):
        idxs = np.random.randint(0,
                                 self.capacity if self.full else self.idx,
                                 size=batch_size)
        obses = torch.as_tensor(self.obses[idxs], device=self.device).float()
        actions = torch.as_tensor(self.actions[idxs], device=self.device)
        rewards = torch.as_tensor(self.rewards[idxs], device=self.device)
        next_obses = torch.as_tensor(self.next_obses[idxs],
                                     device=self.device).float()
        not_dones = torch.as_tensor(self.not_dones[idxs], device=self.device)

        return obses, actions, rewards, next_obses, not_dones


def set_random_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False


class DeterministicDynamicsModel(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_dim=64, hidden_depth=2):
        super(DeterministicDynamicsModel, self).__init__()
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.trunk = mlp(num_inputs, hidden_dim, num_outputs, hidden_depth)

    def forward(self, x):
        v = self.trunk(x)
        v = v + x[:, :v.shape[1]]
        return v

def collect_trajs(
        env,
        agent,
        replay_buffer,
        device,
        episode_length=math.inf,
        render=False,
):
    # Collect the following data
    raw_obs = []
    raw_next_obs = []
    actions = []
    rewards = []
    dones = []
    images = []

    path_length = 0

    o = env.reset()
    if render:
        env.render()

    while path_length < episode_length:
        o_for_agent = o

        action, _, _ = agent(torch.Tensor(o_for_agent).unsqueeze(0).to(device))
        action= action.cpu().detach().numpy()[0]

        # Step the simulation forward
        next_o, r, done, env_info = env.step(copy.deepcopy(action))

        replay_buffer.add(o,
                          action,
                          r,
                          next_o,
                          done)

        # Render the environment
        if render:
            env.render()

        raw_obs.append(o)
        raw_next_obs.append(next_o)
        actions.append(action)
        rewards.append(r)
        dones.append(done)
        path_length += 1
        if done:
            break
        o = next_o

    # Prepare the items to be returned
    observations = np.array(raw_obs)
    next_observations = np.array(raw_next_obs)
    actions = np.array(actions)
    if len(actions.shape) == 1:
        actions = np.expand_dims(actions, 1)
    rewards = np.array(rewards)
    if len(rewards.shape) == 1:
        rewards = rewards.reshape(-1, 1)
    dones = np.array(dones).reshape(-1, 1)

    # Return in the following format
    return dict(
        observations=observations,
        next_observations=next_observations,
        actions=actions,
        rewards=rewards,
        dones=np.array(dones).reshape(-1, 1),
        images=np.array(images)
    )

def mlp(input_dim, hidden_dim, output_dim, hidden_depth, output_mod=None):
    if hidden_depth == 0:
        mods = [nn.Linear(input_dim, output_dim)]
    else:
        mods = [nn.Linear(input_dim, hidden_dim), nn.ReLU(inplace=True)]
        for i in range(hidden_depth - 1):
            mods += [nn.Linear(hidden_dim, hidden_dim), nn.ReLU(inplace=True)]
        mods.append(nn.Linear(hidden_dim, output_dim))
    if output_mod is not None:
        mods.append(output_mod)
    trunk = nn.Sequential(*mods)
    return trunk

def rollout(
        env,
        agent,
        episode_length=math.inf,
        render=False,
):
    # Collect the following data
    raw_obs = []
    raw_next_obs = []
    actions = []
    rewards = []
    dones = []
    images = []

    entropy = None
    log_prob = None
    agent_info = None
    path_length = 0

    o = env.reset()
    if render:
        env.render()

    while path_length < episode_length:
        o_for_agent = o

        action, _, _ = agent(torch.Tensor(o_for_agent).unsqueeze(0).to(device))
        action = action.cpu().detach().numpy()[0]

        # Step the simulation forward
        next_o, r, done, env_info = env.step(copy.deepcopy(action))

        # Render the environment
        if render:
            env.render()

        raw_obs.append(o)
        raw_next_obs.append(next_o)
        actions.append(action)
        rewards.append(r)
        dones.append(done)
        path_length += 1
        if done:
            break
        o = next_o

    # Prepare the items to be returned
    observations = np.array(raw_obs)
    next_observations = np.array(raw_next_obs)
    actions = np.array(actions)
    if len(actions.shape) == 1:
        actions = np.expand_dims(actions, 1)
    rewards = np.array(rewards)
    if len(rewards.shape) == 1:
        rewards = rewards.reshape(-1, 1)
    dones = np.array(dones).reshape(-1, 1)

    # Return in the following format
    return dict(
        observations=observations,
        next_observations=next_observations,
        actions=actions,
        rewards=rewards,
        dones=np.array(dones).reshape(-1, 1),
        images=np.array(images)
    )



In [ ]:
def evaluate(env, model, plan_mode, num_validation_runs=10, episode_length=200, render=False, mpc_horizon=None, n_samples_mpc=None, reward_fn=None):
    success_count = 0
    rewards_suc = 0
    rewards_all = 0
    for k in range(num_validation_runs):
        o = env.reset()
        path = collect_traj_MBRL(
            env,
            model,
            plan_mode,
            episode_length=episode_length,
            render=render,
            mpc_horizon=mpc_horizon,
            n_samples_mpc=n_samples_mpc,
            device=device,
            reward_fn=reward_fn
        )
        success = np.linalg.norm(env.get_body_com("fingertip") - env.get_body_com("target"))<0.1

        if success:
            success_count += 1
            rewards_suc += np.sum(path['rewards'])
        rewards_all += np.sum(path['rewards'])
        print(f"test {k}, success {success}, reward {np.sum(path['rewards'])}")
    print("Success rate: ", success_count/num_validation_runs)
    print("Average reward (success only): ", rewards_suc/max(success_count, 1))
    print("Average reward (all): ", rewards_all/num_validation_runs)

In [ ]:
# -------------------------------------------------- TODOs start from here --------------------------------------------------------------

In [ ]:
loss_fn = torch.nn.MSELoss()

def train_single(num_epochs, num_batches,batch_size, model, optimizer, replay_buffer):
    for epoch in range(num_epochs):

        for i in range(num_batches):
            optimizer.zero_grad()
            t1_observations, t1_actions, _, t1_next_observations, _ = replay_buffer.sample(batch_size)
            oa_in = torch.cat([t1_observations, t1_actions], dim=-1)

            next_o_pred = model(oa_in)
            loss = loss_fn(next_o_pred, t1_next_observations)

            loss.backward()
            optimizer.step()

def train_model(model, replay_buffer, optimizer, num_epochs=500, batch_size=32):
    """
    Train a single model with supervised learning
    """
    idxs = np.array(range(len(replay_buffer)))
    num_batches = len(idxs) // batch_size
    if not isinstance(model, list):
        train_single(num_epochs, num_batches, batch_size, model, optimizer, replay_buffer)

    # TODO START-Ensemble models

    # Hint1: try different batch size for each model
    # hint2: check out how we define optimizer and model for ensemble models. During training, each model should have their individual optimizer and batch size to increase diversity.

    # TODO END




In [ ]:
def plan_model_random_shooting(env, state, ac_size, horizon, model, reward_fn, n_samples_mpc=100):
    # TODO START-random MPC with shooting
    # Hint1: randomly sample actions in the action space
    # Hint2: rollout model based on current state and random action, select the best action that maximize the sum of the reward

    # TODO END
    return best_ac, random_actions[best_ac_idx]


def plan_model_mppi(env, state, ac_size, horizon, model, reward_fn, n_samples_mpc=100, n_iter_mppi=10, gaussian_noise_scales=[1.0, 1.0, 0.5, 0.5, 0.2, 0.2, 0.1, 0.1, 0.01, 0.01]):
    assert len(gaussian_noise_scales) == n_iter_mppi
    # Rolling forward random actions through the model
    state_repeats = torch.from_numpy(np.repeat(state[None], n_samples_mpc, axis=0)).cuda()
    # Sampling random actions in the range of the action space
    random_actions = torch.FloatTensor(n_samples_mpc, horizon, ac_size).uniform_(env.action_space.low[0], env.action_space.high[0]).cuda().float()
    # Rolling forward through the mdoel for horizon steps
    if not isinstance(model, list):
        all_states, all_rewards = rollout_model(model, state_repeats, random_actions, horizon, reward_fn)
    # TODO START-add ensemble MPPI
    # Hint 1: rollout each model and concatenate rewards for each model


    # TODO END



    all_returns = all_rewards.sum(axis=-1)
    # Take first action from best trajectory
    # best_ac_idx = np.argmax(all_rewards.sum(axis=-1))
    # best_ac = random_actions[best_ac_idx, 0] # Take the first action from the best trajectory

    # Run through a few iterations of MPPI

    # TODO START-MPPI
    # Hint1: Compute weights based on exponential of returns
    # Hint2: sample actions based on the weight, and compute average return over models
    # Hint3: if model type is a list, then implement ensemble mppi




    # TODO END

    # Finally take first action from best trajectory
    best_ac_idx = np.argmax(all_rewards.sum(axis=-1))
    best_ac = random_actions[best_ac_idx, 0] # Take the first action from the best trajectory
    return best_ac, random_actions[best_ac_idx]


def rollout_model(
        model,
        initial_states,
        actions,
        horizon,
        reward_fn):
    # Collect the following data
    all_states = []
    all_rewards = []
    curr_state = initial_states # Starting from the initial state
    # TODO START

    # Hint1: concatenate current state and action pairs as the input for the model and predict the next observation
    # Hint2: get the predicted reward using reward_fn()

    # TODO END
    all_states_full = torch.cat([state[:, None, :] for state in all_states], dim=1).cpu().detach().numpy()
    all_rewards_full = torch.cat(all_rewards, dim=-1).cpu().detach().numpy()
    return all_states_full, all_rewards_full

def planning_agent(env, o_for_agent, model, reward_fn, plan_mode, mpc_horizon=None, n_samples_mpc=None):
    if plan_mode == 'random':
        # Taking random actions
        action = torch.Tensor(env.action_space.sample()[None]).cuda()
    elif plan_mode == 'random_mpc':
        # Taking actions via random shooting + MPC
        action, _ = plan_model_random_shooting(env, o_for_agent, env.action_space.shape[0], mpc_horizon, model,
                                               reward_fn, n_samples_mpc=n_samples_mpc)
    elif plan_mode == 'mppi':
        action, _ = plan_model_mppi(env, o_for_agent, env.action_space.shape[0], mpc_horizon, model, reward_fn,
                                    n_samples_mpc=n_samples_mpc)
    else:
        raise NotImplementedError("Other planning methods not implemented")
    return action

def collect_traj_MBRL(
        env,
        model,
        plan_mode,
        replay_buffer=None,
        device='cuda:0',
        episode_length=math.inf,
        reward_fn=None, #Reward function to evaluate
        render=False,
        mpc_horizon=None,
        n_samples_mpc=None
):
    # Collect the following data
    raw_obs = []
    raw_next_obs = []
    actions = []
    rewards = []
    dones = []
    images = []

    path_length = 0
    o = env.reset()
    if render:
        env.render()

    while path_length < episode_length:
        o_for_agent = o

        # Using the planning agent to take actions
        action = planning_agent(env, o_for_agent, model, reward_fn, plan_mode, mpc_horizon=mpc_horizon, n_samples_mpc=n_samples_mpc)
        action= action.cpu().detach().numpy()[0]

        # Step the simulation forward
        next_o, r, done, env_info = env.step(copy.deepcopy(action))
        if replay_buffer is not None:
            replay_buffer.add(o,
                            action,
                            r,
                            next_o,
                            done)
        # Render the environment
        if render:
            env.render()

        raw_obs.append(o)
        raw_next_obs.append(next_o)
        actions.append(action)
        rewards.append(r)
        dones.append(done)
        path_length += 1
        if done:
            break
        o = next_o

    # Prepare the items to be returned
    observations = np.array(raw_obs)
    next_observations = np.array(raw_next_obs)
    actions = np.array(actions)
    if len(actions.shape) == 1:
        actions = np.expand_dims(actions, 1)
    rewards = np.array(rewards)
    if len(rewards.shape) == 1:
        rewards = rewards.reshape(-1, 1)
    dones = np.array(dones).reshape(-1, 1)

    # Return in the following format
    return dict(
        observations=observations,
        next_observations=next_observations,
        actions=actions,
        rewards=rewards,
        dones=np.array(dones).reshape(-1, 1),
        images=np.array(images)
    )

# Training loop for policy gradient
def simulate_mbrl(env, model, plan_mode, num_epochs=200, max_path_length=200, mpc_horizon=10, n_samples_mpc=200,
                  batch_size=100, num_agent_train_epochs_per_iter=1000, capacity=100000, num_traj_per_iter=100, gamma=0.99, print_freq=10, device = "cuda", reward_fn=None):

    # Set up optimizer and replay buffer
    if not isinstance(model, list):
        optimizer = optim.Adam(model.parameters(), lr=1e-4)

    else:
        print('Initialize separate optimizers for ensemble mbrl')
        # TODO START
        # Hint: try using separate optimizer with different learning rate for each model.
        # TODO END
    replay_buffer = ReplayBuffer(obs_size = env.observation_space.shape[0],
                                 action_size = env.action_space.shape[0],
                                 capacity=capacity,
                                 device=device)

    # Iterate through data collection and planning loop
    for iter_num in range(num_epochs):
        # Sampling trajectories
        sample_trajs = []
        if iter_num == 0:
            # Seed with some initial data, collecting with mode random
            for it in range(num_traj_per_iter):
                sample_traj = collect_traj_MBRL(env=env,
                                                model=model,
                                                plan_mode='random',
                                                replay_buffer=replay_buffer,
                                                device=device,
                                                episode_length=max_path_length,
                                                reward_fn=reward_fn, #Reward function to evaluate
                                                render=False,
                                                mpc_horizon=None,
                                                n_samples_mpc=None)
                sample_trajs.append(sample_traj)
        else:
            for it in range(num_traj_per_iter):
                sample_traj = collect_traj_MBRL(env=env,
                                                model=model,
                                                plan_mode=plan_mode,
                                                replay_buffer=replay_buffer,
                                                device=device,
                                                episode_length=max_path_length,
                                                reward_fn=reward_fn, #Reward function to evaluate
                                                render=False,
                                                mpc_horizon=mpc_horizon,
                                                n_samples_mpc=n_samples_mpc)
                sample_trajs.append(sample_traj)

        # Train the model
        train_model(model, replay_buffer, optimizer, num_epochs=num_agent_train_epochs_per_iter, batch_size=batch_size)

        # Logging returns occasionally
        if iter_num % print_freq == 0:

            rewards_np = np.mean(np.asarray([traj['rewards'].sum() for traj in sample_trajs]))
            path_length = np.max(np.asarray([traj['rewards'].shape[0] for traj in sample_trajs]))
            print("Episode: {}, reward: {}, max path length: {}".format(iter_num, rewards_np, path_length))

In [ ]:
class Args:
    def __init__(self, model_type, plan_mode, test=False, render=False):
        self.model_type = model_type
        self.plan_mode = plan_mode
        self.test = test # whether test only
        self.render = render # whether to render during test
args = Args('single', 'random_mpc', False,  False) # plan_mode choose from ['random_mpc', 'mppi'] and model_type choose from ['single', 'ensemble']

In [ ]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

if __name__ == '__main__':
    if args.render:
        import os
        os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libGLEW.so"

    # Environment and reward definition
    env = gym.make("Reacher-v2")
    def reward_fn_reacher(state, action):
        cos_theta = state[:, :2]
        sin_theta = state[:, 2:4]
        qpos = state[:, 4:6]
        qvel = state[:, 6:8]
        vec = state[:, 8:11]

        reward_dist = -torch.norm(vec, dim=1)
        reward_ctrl = -torch.square(action).sum(dim=1)

        reward = reward_dist + reward_ctrl
        reward = reward[:, None]
        return reward
    reward_fn = reward_fn_reacher

    # Define dynamics model
    hidden_dim_model = 64
    hidden_depth_model = 2
    if args.model_type == 'single':
        model = DeterministicDynamicsModel(env.observation_space.shape[0] + env.action_space.shape[0], env.observation_space.shape[0], hidden_dim=hidden_dim_model, hidden_depth=hidden_depth_model)
        model.to(device)
    elif args.model_type == 'ensemble':
        num_ensembles = 5
        model = []
        for model_id in range(num_ensembles):
            curr_model = DeterministicDynamicsModel(env.observation_space.shape[0] + env.action_space.shape[0], env.observation_space.shape[0], hidden_dim=hidden_dim_model, hidden_depth=hidden_depth_model)
            curr_model.to(device)
            model.append(curr_model)
    else:
        raise NotImplementedError("No other model types implemented")

    # Training hyperparameters
    num_epochs=15
    max_path_length=50
    batch_size=250 #5000
    num_agent_train_epochs_per_iter=10 #100
    num_traj_per_iter = batch_size // max_path_length
    gamma=0.99
    print_freq=1
    capacity=100000
    mpc_horizon = 10
    n_samples_mpc = 1000

    if not args.test:
        # Training and model saving code
        simulate_mbrl(env, model, plan_mode=args.plan_mode, num_epochs=num_epochs, max_path_length=max_path_length, mpc_horizon=mpc_horizon,
                    n_samples_mpc=n_samples_mpc, batch_size=batch_size, num_agent_train_epochs_per_iter=num_agent_train_epochs_per_iter, capacity=capacity, num_traj_per_iter=num_traj_per_iter, gamma=gamma, print_freq=print_freq, device = "cuda", reward_fn=reward_fn)
        if type(model) is list:
            for model_idx, curr_model in enumerate(model):
                torch.save(curr_model.state_dict(), f'{args.model_type}_{args.plan_mode}_{model_idx}.pth')
        else:
            torch.save(model.state_dict(), f'{args.model_type}_{args.plan_mode}.pth')
    else:
        print('loading pretrained mbrl')
        if type(model) is list:
            for model_idx in range(len(model)):

                model[model_idx].load_state_dict(torch.load(f'{args.model_type}_{args.plan_mode}_{model_idx}.pth'))
        else:
            model.load_state_dict(torch.load(f'{args.model_type}_{args.plan_mode}.pth'))

    evaluate(env, model, plan_mode=args.plan_mode, mpc_horizon=mpc_horizon, n_samples_mpc=n_samples_mpc, num_validation_runs=100, episode_length=max_path_length, render=args.render, reward_fn=reward_fn)